# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [6]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [16]:
# your code here
orders = pd.read_csv('orders.csv').drop(['Unnamed: 0'], axis=1)
orders.shape

(397924, 13)

In [17]:
orders.head()

,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [14]:
# your code here
customers = orders.groupby(['CustomerID']).agg({'amount_spent':sum}).reset_index()
customers

,CustomerID,amount_spent
0,12346,77183.60
1,12347,4310.00
2,12348,1797.24
3,12349,1757.55
4,12350,334.40
...,...,...
4334,18280,180.60
4335,18281,80.82
4336,18282,178.05
4337,18283,2094.88


In [15]:
customers.describe().T

,count,mean,std,min,25%,50%,75%,max
CustomerID,4339.0,15299.936852,1721.889758,12346.0,13812.500,15299.00,16778.50,18287.00
amount_spent,4339.0,2053.793018,8988.248381,0.0,307.245,674.45,1661.64,280206.02


In [23]:
def categorize_customer(row):
    if row['amount_spent'] > np.percentile(customers['amount_spent'], 90):
        return 'VIP'
    elif row['amount_spent'] > np.percentile(customers['amount_spent'], 75):
        return 'Preferred'
    else:
        return 'Regular'
    
customers['Category'] = customers.apply(categorize_customer, axis =1)
customers

,CustomerID,amount_spent,Category
0,12346,77183.60,VIP
1,12347,4310.00,VIP
2,12348,1797.24,Preferred
3,12349,1757.55,Preferred
4,12350,334.40,Regular
...,...,...,...
4334,18280,180.60,Regular
4335,18281,80.82,Regular
4336,18282,178.05,Regular
4337,18283,2094.88,Preferred


In [25]:
customers['Category'].value_counts()

Regular      3254
Preferred     651
VIP           434
Name: Category, dtype: int64

Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [28]:
# your code here: 
countries = orders.groupby(['Country', 'CustomerID']).agg({'amount_spent': 'sum'}).reset_index()
countries

,Country,CustomerID,amount_spent
0,Australia,12386,401.90
1,Australia,12388,2780.66
2,Australia,12393,1582.60
3,Australia,12415,124914.53
4,Australia,12422,386.20
...,...,...,...
4342,United Kingdom,18287,1837.28
4343,Unspecified,12363,552.00
4344,Unspecified,12743,546.43
4345,Unspecified,14265,530.18


In [33]:
combined = pd.merge(left = countries, right = customers, how = 'inner', left_on = 'CustomerID', right_on = 'CustomerID')

In [36]:
combined

,Country,CustomerID,amount_spent_x,amount_spent_y,Category
0,Australia,12386,401.90,401.90,Regular
1,Australia,12388,2780.66,2780.66,Preferred
2,Australia,12393,1582.60,1582.60,Regular
3,Australia,12415,124914.53,124914.53,VIP
4,Australia,12422,386.20,803.56,Regular
...,...,...,...,...,...
4342,United Kingdom,18287,1837.28,1837.28,Preferred
4343,Unspecified,12363,552.00,552.00,Regular
4344,Unspecified,12743,546.43,546.43,Regular
4345,Unspecified,14265,530.18,530.18,Regular


In [39]:
combined[combined['Category']=='VIP']['Country'].value_counts()

United Kingdom     354
Germany             22
France              18
Spain                7
Switzerland          5
Norway               4
Belgium              4
Japan                3
EIRE                 3
Portugal             2
Australia            2
Finland              2
Denmark              2
Israel               1
Cyprus               1
Netherlands          1
Channel Islands      1
Poland               1
Singapore            1
Sweden               1
Austria              1
Iceland              1
Name: Country, dtype: int64

## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [41]:
# your code here
# combined[(combined['Category']=='VIP')|(combined['Category']=='Preferred')]['Country'].value_counts()

combined[(combined['Category']=='VIP') | combined['Category']=='Preferred')]['Country'].value_counts()

SyntaxError: closing parenthesis ')' does not match opening parenthesis '[' (3052103747.py, line 4)